In [2]:
!pip install wandb

In [4]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import argparse, os
import wandb
from pathlib import Path
import torchvision.models as tvmodels
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
from sklearn.metrics import ConfusionMatrixDisplay
from IPython.display import display, Markdown

WANDB_PROJECT = "mlops-course-001"
ENTITY = None
BDD_CLASSES = {i:c for i,c in enumerate(['background', 'road', 'traffic light', 'traffic sign', 'person', 'vehicle', 'bicycle'])}
RAW_DATA_AT = 'av-team/mlops-course-001/bdd_simple_1k'
PROCESSED_DATA_AT = 'av-team/mlops-course-001/bdd_simple_1k_split'

CLASS_INDEX = {v:k for k,v in BDD_CLASSES.items()}

def t_or_f(arg):
    ua = str(arg).upper()
    if 'TRUE'.startswith(ua): return True
    else: return False

def iou_per_class(inp, targ):
    "Compute iou per class"
    iou_scores = []
    for c in range(inp.shape[0]):
        dec_preds = inp.argmax(dim=0)
        p = torch.where(dec_preds == c, 1, 0)
        t = torch.where(targ == c, 1, 0)
        c_inter = (p * t).float().sum().item()
        c_union = (p + t).float().sum().item()
        iou_scores.append(c_inter / (c_union - c_inter) if c_union > 0 else np.nan)
    return iou_scores

def create_row(sample, pred_label, prediction, class_labels):
    """"A simple function to create a row of (img, target, prediction, and scores...)"""
    (image, label) = sample
    # compute metrics
    iou_scores = iou_per_class(prediction, label)
    image = image.permute(1, 2, 0)
    row =[wandb.Image(
                image,
                masks={
                    "predictions": {
                        "mask_data": pred_label[0].numpy(),
                        "class_labels": class_labels,
                    },
                    "ground_truths": {
                        "mask_data": label.numpy(),
                        "class_labels": class_labels,
                    },
                },
            ),
            *iou_scores,
    ]
    return row

def create_iou_table(samples, outputs, predictions, class_labels):
    "Creates a wandb table with predictions and targets side by side"

    def _to_str(l):
        return [f'{str(x)} IoU' for x in l]

    items = list(zip(samples, outputs, predictions))

    table = wandb.Table(
        columns=["Image"]
        + _to_str(class_labels.values()),
    )
    # we create one row per sample
    for item in progress_bar(items):
        table.add_data(*create_row(*item, class_labels=class_labels))

    return table

def get_predictions(learner, test_dl=None, max_n=None):
    """Return the samples = (x,y) and outputs (model predictions decoded), and predictions (raw preds)"""
    test_dl = learner.dls.valid if test_dl is None else test_dl
    inputs, predictions, targets, outputs = learner.get_preds(
        dl=test_dl, with_input=True, with_decoded=True
    )
    x, y, samples, outputs = learner.dls.valid.show_results(
        tuplify(inputs) + tuplify(targets), outputs, show=False, max_n=max_n
    )
    return samples, outputs, predictions

    def value(self): return self.inter/(self.union-self.inter) if self.union > 0 else None

class MIOU(DiceMulti):
    @property
    def value(self):
        binary_iou_scores = np.array([])
        for c in self.inter:
            binary_iou_scores = np.append(binary_iou_scores, \
                                          self.inter[c]/(self.union[c]-self.inter[c]) if self.union[c] > 0 else np.nan)
        return np.nanmean(binary_iou_scores)

class IOU(DiceMulti):
    @property
    def value(self):
        c=CLASS_INDEX[self.nm]
        return self.inter[c]/(self.union[c]-self.inter[c]) if self.union[c] > 0 else np.nan

class BackgroundIOU(IOU): nm = 'background'
class RoadIOU(IOU): nm = 'road'
class TrafficLightIOU(IOU): nm = 'traffic light'
class TrafficSignIOU(IOU): nm = 'traffic sign'
class PersonIOU(IOU): nm = 'person'
class VehicleIOU(IOU): nm = 'vehicle'
class BicycleIOU(IOU): nm = 'bicycle'


class IOUMacro(DiceMulti):
    @property
    def value(self):
        c=CLASS_INDEX[self.nm]
        if c not in self.count: return np.nan
        else: return self.macro[c]/self.count[c] if self.count[c] > 0 else np.nan

    def reset(self): self.macro,self.count = {},{}

    def accumulate(self, learn):
        pred,targ = learn.pred.argmax(dim=self.axis), learn.y
        for c in range(learn.pred.shape[self.axis]):
            p = torch.where(pred == c, 1, 0)
            t = torch.where(targ == c, 1, 0)
            c_inter = (p*t).float().sum(dim=(1,2))
            c_union = (p+t).float().sum(dim=(1,2))
            m = c_inter / (c_union - c_inter)
            macro = m[~torch.any(m.isnan())]
            count = macro.shape[1]

            if count > 0:
                msum = macro.sum().item()
                if c in self.count:
                    self.count[c] += count
                    self.macro[c] += msum
                else:
                    self.count[c] = count
                    self.macro[c] = msum


class MIouMacro(IOUMacro):
    @property
    def value(self):
        binary_iou_scores = np.array([])
        for c in self.count:
            binary_iou_scores = np.append(binary_iou_scores, self.macro[c]/self.count[c] if self.count[c] > 0 else np.nan)
        return np.nanmean(binary_iou_scores)


class BackgroundIouMacro(IOUMacro): nm = 'background'
class RoadIouMacro(IOUMacro): nm = 'road'
class TrafficLightIouMacro(IOUMacro): nm = 'traffic light'
class TrafficSignIouMacro(IOUMacro): nm = 'traffic sign'
class PersonIouMacro(IOUMacro): nm = 'person'
class VehicleIouMacro(IOUMacro): nm = 'vehicle'
class BicycleIouMacro(IOUMacro): nm = 'bicycle'


def display_diagnostics(learner, dls=None, return_vals=False):
    """
    Display a confusion matrix for the unet learner.
    If `dls` is None it will get the validation set from the Learner

    You can create a test dataloader using the `test_dl()` method like so:
    >> dls = ... # You usually create this from the DataBlocks api, in this library it is get_data()
    >> tdls = dls.test_dl(test_dataframe, with_labels=True)

    See: https://docs.fast.ai/tutorial.pets.html#adding-a-test-dataloader-for-inference

    """
    probs, targs = learner.get_preds(dl = dls)
    preds = probs.argmax(dim=1)
    classes = list(BDD_CLASSES.values())
    y_true = targs.flatten().numpy()
    y_pred = preds.flatten().numpy()

    tdf, pdf = [pd.DataFrame(r).value_counts().to_frame(c) for r,c in zip((y_true, y_pred) , ['y_true', 'y_pred'])]
    countdf = tdf.join(pdf, how='outer').reset_index(drop=True).fillna(0).astype(int).rename(index=BDD_CLASSES)
    countdf = countdf/countdf.sum()
    display(Markdown('### % Of Pixels In Each Class'))
    display(countdf.style.format('{:.1%}'))


    disp = ConfusionMatrixDisplay.from_predictions(y_true=y_true, y_pred=y_pred,
                                                   display_labels=classes,
                                                   normalize='pred')
    fig = disp.ax_.get_figure()
    fig.set_figwidth(10)
    fig.set_figheight(10)
    disp.ax_.set_title('Confusion Matrix (by Pixels)', fontdict={'fontsize': 32, 'fontweight': 'medium'})
    fig.show()

    if return_vals: return countdf, disp

default_config = SimpleNamespace(
    framework="fastai",
    img_size=180,
    batch_size=8,
    augment=True,
    epochs=10,
    lr=2e-3,
    pretrained=True,
    mixed_precision=True,
    arch="resnet18",
    seed=42,
    log_preds=False,
)


def parse_args():
    "Overriding default argments"
    argparser = argparse.ArgumentParser(description='Process hyper-parameters')
    argparser.add_argument('--img_size', type=int, default=default_config.img_size, help='image size')
    argparser.add_argument('--batch_size', type=int, default=default_config.batch_size, help='batch size')
    argparser.add_argument('--epochs', type=int, default=default_config.epochs, help='number of training epochs')
    argparser.add_argument('--lr', type=float, default=default_config.lr, help='learning rate')
    argparser.add_argument('--arch', type=str, default=default_config.arch, help='timm backbone architecture')
    argparser.add_argument('--augment', type=t_or_f, default=default_config.augment, help='Use image augmentation')
    argparser.add_argument('--seed', type=int, default=default_config.seed, help='random seed')
    argparser.add_argument('--log_preds', type=t_or_f, default=default_config.log_preds, help='log model predictions')
    argparser.add_argument('--pretrained', type=t_or_f, default=default_config.pretrained, help='Use pretrained model')
    argparser.add_argument('--mixed_precision', type=t_or_f, default=default_config.mixed_precision, help='use fp16')
    args = argparser.parse_args()
    vars(default_config).update(vars(args))
    return

def download_data():
    "Grab dataset from artifact"
    processed_data_at = wandb.use_artifact(f'{PROCESSED_DATA_AT}:latest')
    processed_dataset_dir = Path(processed_data_at.download())
    return processed_dataset_dir

def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_df(processed_dataset_dir, is_test=False):
    df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

    if not is_test:
        df = df[df.Stage != 'test'].reset_index(drop=True)
        df['is_valid'] = df.Stage == 'valid'
    else:
        df = df[df.Stage == 'test'].reset_index(drop=True)

    df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
    df["label_fname"] = [label_func(f) for f in df.image_fname.values]
    return df

def get_data(df, bs=4, img_size=180, augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize((img_size, int(img_size * 16 / 9))),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)


def log_predictions(learn):
    "Log a Table with model predictions and metrics"
    samples, outputs, predictions = get_predictions(learn)
    table = create_iou_table(samples, outputs, predictions, BDD_CLASSES)
    wandb.log({"pred_table":table})

def final_metrics(learn):
    "Log latest metrics values"
    scores = learn.validate()
    metric_names = ['final_loss'] + [f'final_{x.name}' for x in learn.metrics]
    final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
    for k,v in final_results.items():
        wandb.summary[k] = v

def train(config):
    set_seed(config.seed)
    run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="training", config=config)

    config = wandb.config

    processed_dataset_dir = download_data()
    proc_df = get_df(processed_dataset_dir)
    dls = get_data(proc_df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

    metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(),
               TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

    cbs = [WandbCallback(log_preds=False, log_model=True),
           SaveModelCallback(fname=f'run-{wandb.run.id}-model', monitor='miou')]
    cbs += ([MixedPrecision()] if config.mixed_precision else [])

    learn = unet_learner(dls, arch=getattr(tvmodels, config.arch), pretrained=config.pretrained,
                         metrics=metrics)

    learn.fit_one_cycle(config.epochs, config.lr, cbs=cbs)
    if config.log_preds:
        log_predictions(learn)
    final_metrics(learn)

    wandb.finish()

if __name__ == '__main__':
    train(default_config)

wandb: Currently logged in as: semaljohari80. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:1:11.7
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 205MB/s]
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.500284,0.357994,0.300947,0.850211,0.633466,0.000000,0.000000,0.000000,0.622953,0.000000,00:30
1,0.446574,0.318922,0.324645,0.876110,0.771224,0.000000,0.000000,0.000000,0.625179,0.000000,00:27
2,0.349148,0.330045,0.335540,0.896977,0.797141,0.000000,0.000000,0.000000,0.654661,0.000000,00:28
3,0.317041,0.275561,0.341716,0.900212,0.816845,0.000000,0.000000,0.000000,0.674955,0.000000,00:27
4,0.283170,0.270434,0.344577,0.904826,0.806344,0.000000,0.000000,0.000000,0.700870,0.000000,00:30
5,0.261762,0.248178,0.349732,0.909839,0.833555,0.000000,0.000000,0.000000,0.704728,0.000000,00:28
6,0.225238,0.252865,0.348215,0.909293,0.824481,0.001589,0.000000,0.000000,0.702143,0.000000,00:28
7,0.214730,0.229136,0.360176,0.916082,0.842138,0.025439,0.000000,0.000000,0.737573,0.000000,00:27
8,0.192772,0.233366,0.368158,0.917028,0.840758,0.070894,0.000000,0.000000,0.748425,0.000000,00:28
9,0.181197,0.228133,0.366658,0.917972,0.843421,0.056114,0.000000,0.000000,0.749101,0.000000,00:30


Better model found at epoch 0 with miou value: 0.30094723380501337.
Better model found at epoch 1 with miou value: 0.3246446708863483.
Better model found at epoch 2 with miou value: 0.3355398223554365.
Better model found at epoch 3 with miou value: 0.34171598255395624.
Better model found at epoch 4 with miou value: 0.3445771476645323.
Better model found at epoch 5 with miou value: 0.3497318810849751.
Better model found at epoch 7 with miou value: 0.3601760311625872.
Better model found at epoch 8 with miou value: 0.36815792148425774.


/usr/local/lib/python3.10/dist-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **torch_load_kwargs

background_iou,▁▄▆▆▇▇▇███
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▂▂▃▃▄▅▆█████▇▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
lr_1,▁▁▂▂▃▄▄▄▆▇██████▇▇▇▇▇▆▆▆▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁
lr_2,▁▅▅▅▆█████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
miou,▁▃▅▅▆▆▆▇██
mom_0,███▆▆▅▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇▇▇███


In [7]:
sweep_id = "semaljohari80/mlops-course-001/y6hpxrsx"
wandb.agent(sweep_id, function=lambda: train(default_config), count=5)

wandb: Agent Starting Run: cwyak877 with config:
wandb: 	arch: mobilenet_v3_small
wandb: 	batch_size: 4
wandb: 	img_size: 240
wandb: 	log_preds: False
wandb: 	lr: 0.007991361019072305
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:11.3
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 121MB/s]
/usr/loc

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.499907,0.511631,0.232469,0.796517,0.513958,0.000000,0.000000,0.000000,0.316807,0.000000,03:53
1,0.535532,0.639390,0.204978,0.760867,0.515678,0.000000,0.000000,0.000000,0.158303,0.000000,03:49
2,0.435439,0.472977,0.256498,0.814860,0.573803,0.000000,0.000000,0.000000,0.406823,0.000000,03:47
3,0.370528,0.452720,0.250415,0.807845,0.617090,0.000000,0.000000,0.000000,0.327969,0.000000,03:45
4,0.342067,0.343425,0.299869,0.862733,0.714663,0.000000,0.000000,0.000000,0.521689,0.000000,03:45
5,0.324486,0.359898,0.297818,0.856997,0.748921,0.000000,0.000000,0.000000,0.478807,0.000000,03:45
6,0.288434,0.322625,0.314939,0.871597,0.747208,0.000000,0.000000,0.000000,0.585770,0.000000,03:44
7,0.268823,0.293830,0.323862,0.886783,0.774071,0.000000,0.000000,0.000000,0.606182,0.000000,03:45
8,0.247600,0.290471,0.327905,0.890318,0.779071,0.000000,0.000000,0.000000,0.625947,0.000000,03:45
9,0.241062,0.285981,0.329302,0.892324,0.783020,0.000000,0.000000,0.000000,0.629772,0.000000,03:44


Better model found at epoch 0 with miou value: 0.2324687789755749.
Better model found at epoch 2 with miou value: 0.25649806720129603.
Better model found at epoch 4 with miou value: 0.29986924730411657.
Better model found at epoch 6 with miou value: 0.3149392698542037.
Better model found at epoch 7 with miou value: 0.323862310368059.
Better model found at epoch 8 with miou value: 0.32790506032419614.
Better model found at epoch 9 with miou value: 0.32930223330883274.


/usr/local/lib/python3.10/dist-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **torch_load_kwargs

...

wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:10.6
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_X_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_X_400MF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/regnet_x_400mf-adf1edd5.pth" to /root/.cache/torch/hub/checkpoints/regnet_x_400mf-adf1edd5.pth
100%|██████████| 21.3M/21.3M [00:00<00:00, 163MB/s]
/usr/local/lib/python3.1

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.539174,0.391158,0.281817,0.841793,0.649878,0.000000,0.000000,0.000000,0.481047,0.000000,00:45
1,0.423147,0.338105,0.316604,0.879525,0.743858,0.000000,0.000000,0.000000,0.592847,0.000000,00:45
2,0.352425,0.318153,0.328037,0.889311,0.767716,0.000000,0.000000,0.000000,0.639230,0.000000,00:45
3,0.309379,0.279868,0.336298,0.894118,0.781254,0.000245,0.000000,0.000000,0.678468,0.000000,00:44
4,0.278287,0.270891,0.337119,0.897468,0.780215,0.000000,0.000000,0.000000,0.682148,0.000000,00:43
5,0.260320,0.261985,0.343944,0.900623,0.789380,0.036209,0.000000,0.000000,0.681393,0.000000,00:44
6,0.230090,0.275780,0.342286,0.902005,0.802021,0.000163,0.000000,0.000000,0.691812,0.000000,00:44
7,0.217683,0.246902,0.359463,0.907276,0.801541,0.050168,0.037036,0.000000,0.720220,0.000000,00:44
8,0.196771,0.250374,0.360433,0.909525,0.810941,0.039291,0.035447,0.000000,0.727825,0.000000,00:43
9,0.189627,0.248097,0.362384,0.910734,0.813120,0.048072,0.037046,0.000257,0.727457,0.000000,00:44


Better model found at epoch 0 with miou value: 0.28181673353936737.
Better model found at epoch 1 with miou value: 0.31660435254883856.
Better model found at epoch 2 with miou value: 0.32803674783241027.
Better model found at epoch 3 with miou value: 0.3362979565805576.
Better model found at epoch 4 with miou value: 0.3371186868530038.
Better model found at epoch 5 with miou value: 0.3439435515044101.
Better model found at epoch 7 with miou value: 0.35946286675764355.
Better model found at epoch 8 with miou value: 0.3604328455706071.
Better model found at epoch 9 with miou value: 0.36238371683562426.


/usr/local/lib/python3.10/dist-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **torch_load_kwargs

background_iou,▁▅▆▆▇▇▇███
bicycle_iou,▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇████
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▅▇██████████▇▇▇▇▆▆▅▅▅▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁
lr_1,▁▁▂▂▃▆▆▇██████████▇▇▇▆▆▆▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▁
miou,▁▄▅▆▆▆▆███
mom_0,█▇▇▇▅▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
mom_1,████▇▅▅▄▃▂▁▁▁▁▂▂▂▂▂▂▃▄▄▄▄▅▆▆▆▆▆▇▇▇▇█████
person_iou,▁▁▁▁▁▁▁▁▁█
